In [69]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements, solve_ritz_two_vec
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, coeff_gram_schmidt
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from functools import reduce


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [70]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [71]:
#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# e, v = sparse.linalg.eigsh(H_correct_10e, which = 'SA', k = 10)
# e_ground = e[0]
# e_1st_ex = e[1]

e_ground = -84.92262983120877
e_1st_ex = -84.52780562388604
e_2nd_ex = -84.46820215626565
e_3rd_ex = -84.42486181064207
e_4th_ex = -84.42379006099287

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [72]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 58, 96, 96, 58, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [73]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [74]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])
HFS2 = generate_single_state(7, [3, 3, 3, 3, 1, 2, 0])
HFS3 = generate_single_state(7, [3, 3, 3, 3, 1, 0, 2])
HFS4 = generate_single_state(7, [3, 3, 3, 3, 0, 1, 2])

initial = add_mps(HFS4, add_mps(HFS3, add_mps(HFS, HFS2)))
initial.orthonormalize('right')

2.0

We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [75]:
N_Krylov = 50
psi_original = copy.deepcopy(initial)
max_bond_Krylov = 80
#max_bond_Krylov = 75
trunc_tol = 1e-12
foldername = f"/work_fast/ge49cag/code_datas/water_Krylov_mixed_initial_1"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

[1, 4, 11, 22, 24, 16, 4, 1]
2
[1, 4, 16, 31, 31, 16, 4, 1]
3
[1, 4, 16, 31, 31, 16, 4, 1]
4
[1, 4, 16, 31, 31, 16, 4, 1]
5
[1, 4, 16, 31, 31, 16, 4, 1]
6
[1, 4, 16, 31, 31, 16, 4, 1]
7
[1, 4, 16, 31, 31, 16, 4, 1]
8
[1, 4, 16, 31, 31, 16, 4, 1]
9
[1, 4, 16, 31, 31, 16, 4, 1]
10
[1, 4, 16, 31, 31, 16, 4, 1]
11
[1, 4, 16, 31, 31, 16, 4, 1]
12
[1, 4, 16, 31, 31, 16, 4, 1]
13
[1, 4, 16, 31, 31, 16, 4, 1]
14
[1, 4, 16, 31, 31, 16, 4, 1]
15
[1, 4, 16, 31, 31, 16, 4, 1]
16
[1, 4, 16, 31, 31, 16, 4, 1]
17
[1, 4, 16, 31, 31, 16, 4, 1]
18
[1, 4, 16, 31, 31, 16, 4, 1]
19
[1, 4, 16, 31, 31, 16, 4, 1]
20
[1, 4, 16, 31, 31, 16, 4, 1]
21
[1, 4, 16, 31, 31, 16, 4, 1]
22
[1, 4, 16, 31, 31, 16, 4, 1]
23
[1, 4, 16, 31, 31, 16, 4, 1]
24
[1, 4, 16, 31, 31, 16, 4, 1]
25
[1, 4, 16, 31, 31, 16, 4, 1]
26
[1, 4, 16, 31, 31, 16, 4, 1]
27
[1, 4, 16, 31, 31, 16, 4, 1]
28
[1, 4, 16, 31, 31, 16, 4, 1]
29
[1, 4, 16, 31, 31, 16, 4, 1]
30
[1, 4, 16, 31, 31, 16, 4, 1]
31
[1, 4, 16, 31, 31, 16, 4, 1]
32
[1, 4, 16, 31, 3

In [76]:
#16m59.2s
#17m19.2s

In [77]:
filename = foldername + f"/Krylov_vec{10}.pkl"
with open(filename, 'rb') as file:
    temp1 = pickle.load(file)
print(len(temp1.qD[0]))

1


In [78]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov, foldername, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov, foldername)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)
H_reduced_1 = remain_only_tridiagonal_elements(H_reduced_1)
e_ritz_non_ortho_1, v_ritz_non_ortho_1 = np.linalg.eigh(H_reduced_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [79]:
np.linalg.eigh(H_reduced_1)

EighResult(eigenvalues=array([-84.92262983, -84.52779309, -84.46813102, -84.32600511,
       -83.9239087 , -83.82603841, -83.52279215, -83.3282669 ,
       -83.0318878 , -82.71637717, -82.40934842, -82.15037203,
       -81.79655212, -81.5262178 , -81.18610674, -80.86294623,
       -80.66173852, -80.42711088, -80.22588966, -66.86802488,
       -64.75156353, -64.42493689, -64.19690043, -63.84654954,
       -63.65406565, -63.16161477, -62.90583201, -62.29131067,
       -61.92983847, -61.53101201, -61.12560054, -60.90095137,
       -60.37930113, -60.31963595, -41.16139517, -41.15136227,
       -40.68690184, -39.92309974, -39.83351356, -39.77404454,
       -39.12527925, -38.98648313, -38.50136841, -38.44987143,
       -38.28839673, -37.83656927, -37.75846939, -37.47014853,
       -37.43163448, -36.75208794]), eigenvectors=array([[ 1.45582354e-01, -2.41749935e-01,  2.49502831e-01, ...,
         3.87115058e-06,  7.68385974e-06, -1.62377713e-05],
       [-2.56564505e-01,  3.27379602e-01, -3.22

In [80]:
e_ritz_1, v_ritz_1 = solve_ritz_two_vec(foldername, H_reduced_1, N_Krylov, coeff_1, max_bond_Krylov, e_ground, e_1st_ex, mpo_ref)

ground error (0.5843924514549741+0j)
first excited error (35.865342370965436+0j)
ground error (0.250913137624849+0j)
first excited error (49.12244284040395+0j)
ground error (0.05222850773203902+0j)
first excited error (70.41974974916073+0j)
ground error (0.001982556276232117+0j)
first excited error (74.08424345995435+0j)
ground error (0.00020261234391227845+0j)
first excited error (74.64050442158798+0j)
ground error (1.2040741239616182e-05+0j)
first excited error (76.60497352013564+0j)
ground error (1.1375683044434481e-06+0j)
first excited error (78.17494367205754+0j)
ground error (6.314058964562719e-08+0j)
first excited error (79.21852449386631+0j)
ground error (1.8872867713071173e-09+0j)
first excited error (79.62125342104024+0j)
ground error (3.879563337250147e-11+0j)
first excited error (79.6643123383523+0j)


In [81]:
# ground error (0.5844515227254732+0j)
# first excited error (35.86758995085174+0j)
# ground error (0.2509182248737005+0j)
# first excited error (49.1137997216985+0j)
# ground error (0.05222820889922275+0j)
# first excited error (70.41936309024723+0j)
# ground error (0.001983361254360716+0j)
# first excited error (74.08655697164855+0j)

In [82]:
shift_1 = e_ritz_1[0]

T_1_shifted = H_reduced_1 - shift_1*np.eye(H_reduced_1.shape[0])

Q, R = np.linalg.qr(T_1_shifted)




In [83]:
array_list = []
for i in range (N_Krylov):
    filename = foldername + f"/Krylov_vec{i}.pkl"
    with open(filename, 'rb') as file:
        vec_array = pickle.load(file)
        vec = vec_array.as_vector()
        vec /= norm(vec)
    array_list.append(vec)

In [84]:
array_list_new = []
for i in range (N_Krylov):
    #temp = Q_total_1[i,0] *array_list[0]
    temp = Q[0,i] *array_list[0]
    for j in range (1, N_Krylov, 1):
        temp = temp + Q[j,i] *array_list[j]
        #temp = temp + Q_total_1[i,j] *array_list[j]
    temp /= norm(temp)
    array_list_new.append(temp)
    
array_list_new = array_list_new[1:20]

In [85]:
len(array_list_new)

19

In [86]:
#array_list_restart_1 = []

for i in range(49,50,1):
    
    temp = H_correct_10e @ array_list_new[i-1]
    #temp /= norm(temp)
    
    overlap1 = np.vdot(array_list_new[i-1], temp)
    overlap2 = np.vdot(array_list_new[i-2], temp)
    
    temp = temp - overlap1* array_list_new[i-1] - overlap2* array_list_new[i-2] 
    #temp = temp - overlap2* array_list_new[i-2]
    
    temp /= norm(temp)

    array_list_new.append(temp)

IndexError: list index out of range

In [ ]:
H_reduced_new = np.zeros([len(array_list_new), len(array_list_new)])
for i in range (len(array_list_new)):
    for j in range (len(array_list_new)):
        if abs(i-j) < 2:
            H_reduced_new[i, j] = np.vdot(array_list_new[i], H_correct_10e @ array_list_new[j])


/tmp/ipykernel_607356/3818840465.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced_new[i, j] = np.vdot(array_list_new[i], H_correct_10e @ array_list_new[j])


In [ ]:
e_test, v_test = np.linalg.eigh(H_reduced_new)

In [ ]:
e_test

array([-84.92090675, -84.48076639, -83.78985361, -83.06564808,
       -82.0785709 , -81.16106025, -80.31244944, -65.06186384,
       -64.80260818, -64.7765909 , -64.01503714, -63.05423236,
       -61.62629797, -60.66948486, -41.15024702, -40.60658173,
       -39.48245102, -38.68907202, -37.79546399, -36.75504906])

In [ ]:
#T_1 = H_reduced_1
Q_list_1 = []
for i in range (len(shift_1)):

    if i == 0:
        T_1 = H_reduced_1
    else: 
        T_1 = R@Q + shift_1[i]*np.eye(T_1.shape[0])

    T_1_shifted = T_1 - shift_1[i]*np.eye(T_1.shape[0])

    Q, R = np.linalg.qr(T_1_shifted)
    Q_list_1.append(Q)

for i in range (len(Q_list_1)):
    if i == 0:
        Q_total_1 = Q_list_1[len(Q_list_1)-2-i] @ Q_list_1[len(Q_list_1)-1-i]
    else:
        Q_total_1 = Q_list_1[len(Q_list_1)-1-i] @ Q_total_1 


IndexError: invalid index to scalar variable.

Continue: expand the space by apply H on mps

In [ ]:
array_list = []
for i in range (N_Krylov_1):
    filename = foldername_1 + f"/Krylov_vec{i}.pkl"
    with open(filename, 'rb') as file:
        vec_array = pickle.load(file)
        vec = vec_array.as_vector()
        vec /= norm(vec)
    array_list.append(vec)

In [ ]:
# for i in range(30):
#     for j in range(30):
#         if i != j:
#             print(np.vdot(array_list[i], array_list[j]))

In [ ]:
array_list_new = []
for i in range (N_Krylov_1):
    #temp = Q_total_1[i,0] *array_list[0]
    temp = Q_total_1[0,i] *array_list[0]
    for j in range (1, N_Krylov_1, 1):
        temp = temp + Q_total_1[j,i] *array_list[j]
        #temp = temp + Q_total_1[i,j] *array_list[j]
    temp /= norm(temp)
    array_list_new.append(temp)

In [ ]:
# for i in range(30):
#     for j in range(30):
#         if i != j:
#             print(np.vdot(array_list_new[i], array_list_new[j]))

In [ ]:
array_list_new = array_list_new[:-10]

In [ ]:
len(array_list_new)

20

In [ ]:
#array_list_restart_1 = []

for i in range(20,30,1):
    
    temp = H_correct_10e @ array_list_new[i-1]
    #temp /= norm(temp)
    
    overlap1 = np.vdot(array_list_new[i-1], temp)
    overlap2 = np.vdot(array_list_new[i-2], temp)
    
    temp = temp - overlap1* array_list_new[i-1] - overlap2* array_list_new[i-2] 
    #temp = temp - overlap2* array_list_new[i-2]
    
    temp /= norm(temp)

    array_list_new.append(temp)

In [ ]:
for i in range(30):
    for j in range(30):
        if i != j:
            print(i, j)
            print(np.vdot(array_list_new[i], array_list_new[j]))

0 1
(-2.3974641716328904e-12+0j)
0 2
(5.856863605213647e-12+0j)
0 3
(-1.3160800574341103e-12+0j)
0 4
(-7.377413567197233e-13+0j)
0 5
(-3.1123675685307273e-12+0j)
0 6
(-5.038599745765815e-13+0j)
0 7
(-4.924739435696601e-12+0j)
0 8
(-5.016517583278368e-12+0j)
0 9
(6.933620344540259e-13+0j)
0 10
(-5.531214375409377e-12+0j)
0 11
(-6.667817339933713e-13+0j)
0 12
(-2.2298222296379677e-12+0j)
0 13
(-2.6475783371227024e-12+0j)
0 14
(-6.416347488047425e-13+0j)
0 15
(-5.903603994550366e-12+0j)
0 16
(2.6938043809487944e-12+0j)
0 17
(-8.674050193391292e-12+0j)
0 18
(-9.113388863002392e-12+0j)
0 19
(-3.137781050613353e-12+0j)
0 20
(-2.0700369803700047e-11+0j)
0 21
(-1.2492984534893947e-08+0j)
0 22
(1.7497164384830833e-07+0j)
0 23
(-5.820305450745557e-07+0j)
0 24
(2.078516736829625e-06+0j)
0 25
(-3.844704224088114e-06+0j)
0 26
(4.819741542064389e-06+0j)
0 27
(1.6264220867309893e-05+0j)
0 28
(-5.176461197878984e-05+0j)
0 29
(8.308480736393415e-05+0j)
1 0
(-2.3974641716328904e-12+0j)
1 2
(-6.599304436

In [ ]:
H_reduced_new = np.zeros([len(array_list_new), len(array_list_new)])
for i in range (len(array_list_new)):
    for j in range (len(array_list_new)):
        if abs(i-j) < 2:
            H_reduced_new[i, j] = np.vdot(array_list_new[i], H_correct_10e @ array_list_new[j])


/tmp/ipykernel_3558023/3818840465.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced_new[i, j] = np.vdot(array_list_new[i], H_correct_10e @ array_list_new[j])


In [ ]:
e_rotate, v_rotate = np.linalg.eigh(H_reduced_new)

In [ ]:
v_rotate_ground = v_rotate[:,0][0]* array_list_new[0]
for i in range (1, 30):
    v_rotate_ground += v_rotate[:,0][i]* array_list_new[i]
v_rotate_ground /= norm(v_rotate_ground)
e_new = np.vdot(v_rotate_ground, H_correct_10e@v_rotate_ground)

In [ ]:
e_ground

-84.92262983120877

In [ ]:
np.linalg.eigh(H_reduced_new)

EighResult(eigenvalues=array([-84.70397839, -84.12563451, -83.5882094 , -82.98044622,
       -82.84234913, -82.47244717, -81.79892431, -81.53889208,
       -80.59641677, -78.38588998, -76.85119897, -72.59692597,
       -72.13963444, -71.0193265 , -68.83520414, -66.49653116,
       -64.18985586, -62.88166621, -61.77038646, -61.30358049,
       -48.59608755, -47.03908734, -46.59025967, -42.15080498,
       -41.17292767, -40.98060391, -39.7371756 , -39.65349118,
       -38.54938167, -37.63338555]), eigenvectors=array([[ 6.15942534e-01,  1.25423190e-13, -6.39830174e-01,
         2.52363401e-01,  2.01925702e-11, -3.36501058e-01,
        -7.31754426e-12,  1.85234678e-01,  3.65007266e-13,
        -9.53219089e-05, -8.21936333e-12, -2.63493203e-11,
         1.70368212e-07, -4.75993049e-08,  5.85675943e-08,
        -7.38070969e-10, -2.50035340e-15, -1.60168921e-15,
        -1.07091474e-15, -6.42135549e-16,  4.19070461e-14,
         1.07931800e-13, -7.44559237e-10,  2.01384917e-07,
         1.383

In [ ]:
e_1st_ex

-84.52780562388604

In [ ]:
#try to use exact eigenvalues as shift
#just use exact matrix form to do it